# Frac Production Data Cleaning

In [1]:
# Necessary imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')


### Wells Data

In [2]:
# Read in Wells Data from Colorado and Wyoming 
wells = pd.read_csv('DJHZ6-17-18Wells.CSV')
wells.head()

,API10,API12,API14,Well Name,Well Number,Lease Name,Operator Alias,Reported Operator,Field,County/Parish,...,Quarter Quarter,District,Abstract,Block,Survey,Surface Hole Latitude (NAD27),Surface Hole Longitude (NAD27),Bottom Hole Latitude (NAD27),Bottom Hole Longitude (NAD27),UWI
0,"=""0500506864""","=""050050686400""","=""05005068640000""",KISSLER DAN,2-10,KISSLER DAN,S&D,S & D LLC,CHALICE,ARAPAHOE (CO),...,SWSW,NaN,NaN,NaN,NaN,39.712224,-104.543270,39.712224,-104.543270,"=""05-005-06864-00"""
1,"=""0500506866""","=""050050686600""","=""05005068660000""",STATE-CRAIG,2-16,STATE-CRAIG,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SESW,NaN,NaN,NaN,NaN,39.609786,-104.334593,39.609786,-104.334593,"=""05-005-06866-00"""
2,"=""0500506867""","=""050050686700""","=""05005068670000""",LINNEBUR,1-20,LINNEBUR,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SESE,NaN,NaN,NaN,NaN,39.595196,-104.344133,39.595196,-104.344133,"=""05-005-06867-00"""
3,"=""0500506870""","=""050050687000""","=""05005068700000""",COLUMBINE-STATE,2,COLUMBINE-STATE,AXIS ENERGY,AXIS EXPLORATION LLC,CHALICE,ARAPAHOE (CO),...,SENE,NaN,NaN,NaN,NaN,39.706044,-104.547569,39.706044,-104.547569,"=""05-005-06870-00"""
4,"=""0500506876""","=""050050687600""","=""05005068760000""",STATE-CRAIG,3-16,STATE-CRAIG,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SWSE,NaN,NaN,NaN,NaN,39.609906,-104.330093,39.609906,-104.330093,"=""05-005-06876-00"""


Alright, this is some fairly clean data, but I already see a couple of spots that need cleaning, such as API numbers, and some columns that might need to be dropped because of their lack of data.  Let's start by cleaning the 14-digit API number because I know I will need that as the identifier for the well. I won't need the other API numbers, as those are not in common with other data sources.

In [3]:
# Cleaning API numbers and setting those as the index
wells.API14 = wells.API14.map(lambda x: int(str(x)[2:16]))
wells = wells.set_index(wells.API14)
# Cleaning the township values
wells.Township = wells.Township.map(lambda x: str(x)[2:5])
wells.head()

,API10,API12,API14,Well Name,Well Number,Lease Name,Operator Alias,Reported Operator,Field,County/Parish,...,Quarter Quarter,District,Abstract,Block,Survey,Surface Hole Latitude (NAD27),Surface Hole Longitude (NAD27),Bottom Hole Latitude (NAD27),Bottom Hole Longitude (NAD27),UWI
API14,,,,,,,,,,,,,,,,,,,,,
5005068640000,"=""0500506864""","=""050050686400""",5005068640000,KISSLER DAN,2-10,KISSLER DAN,S&D,S & D LLC,CHALICE,ARAPAHOE (CO),...,SWSW,NaN,NaN,NaN,NaN,39.712224,-104.543270,39.712224,-104.543270,"=""05-005-06864-00"""
5005068660000,"=""0500506866""","=""050050686600""",5005068660000,STATE-CRAIG,2-16,STATE-CRAIG,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SESW,NaN,NaN,NaN,NaN,39.609786,-104.334593,39.609786,-104.334593,"=""05-005-06866-00"""
5005068670000,"=""0500506867""","=""050050686700""",5005068670000,LINNEBUR,1-20,LINNEBUR,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SESE,NaN,NaN,NaN,NaN,39.595196,-104.344133,39.595196,-104.344133,"=""05-005-06867-00"""
5005068700000,"=""0500506870""","=""050050687000""",5005068700000,COLUMBINE-STATE,2,COLUMBINE-STATE,AXIS ENERGY,AXIS EXPLORATION LLC,CHALICE,ARAPAHOE (CO),...,SENE,NaN,NaN,NaN,NaN,39.706044,-104.547569,39.706044,-104.547569,"=""05-005-06870-00"""
5005068760000,"=""0500506876""","=""050050687600""",5005068760000,STATE-CRAIG,3-16,STATE-CRAIG,OVERLAND RESOURCES,OVERLAND RESOURCES LTD,PRONGHORN,ARAPAHOE (CO),...,SWSE,NaN,NaN,NaN,NaN,39.609906,-104.330093,39.609906,-104.330093,"=""05-005-06876-00"""


Let's now check out all of the columns in this data set.

In [4]:
wells.columns

Index(['API10', 'API12', 'API14', 'Well Name', 'Well Number', 'Lease Name',
       'Operator Alias', 'Reported Operator', 'Field', 'County/Parish',
       'Target Formation', 'DI Landing Zone', 'KB Elevation',
       'Ground Elevation', 'Measured Depth (TD)', 'True Vertical Depth',
       'Production Type', 'Well Status', 'Spud Date', 'Completion Date',
       'Drill Type', 'Upper Perforation', 'Lower Perforation',
       'Gross Perforated Interval', 'Horizontal Length',
       'Wellbore Count (API10)', 'Completion Count (API10)',
       'Treatment Job Count (API10)', 'First Prod Date',
       'Start Date (First Treatment Job)', 'End Date (First Treatment Job)',
       'First Test Gas Volume', 'First Test Oil Volume',
       'First Test Water Volume', 'First Test Hours Tested',
       'First Test Production Method', 'First Well Test Date',
       'First Reported Test Type', 'Last Well Test Date', 'Last Test Type',
       'First Test Gas Gravity', 'First Test Oil Gravity', 'First Test G

Columns I'm interested in and an explanation of what they contain:
- API14 - Well ID
- Well Name - Name of the well
- Well Number - Number of well name
- Operator Alias - Company Name
- Field - General location
- Target Formation - Type of Rock that is being drilled/fraced
- Measured Depth (TD) - Total depth of well (vertical + horizontal distance)
- True Vertical Depth - Vertical depth of well
- Drill Type - Type of well (vertical, horizontal, or distributed)
- Upper Perforation - Depth of first perforation (mostly for vertical wells)
- Lower Perforation - Depth of last perforation (mostly for horizontal wells)
- Gross Perforated Interval - How frequent the cement was perforated 
- Horizontal Length - Length of the horizontal portion of the well
- Treatment Job Count (API10) - The number of times the well was fraced
- Cum BOE - Cumulative Barrels of Oil Equivalent (from oil and gas produced) 
- Cum Gas - Cumulative gas produced
- Cum Oil - Cumulative oil produced
- First 6 BOE - Barrels of oil equvalent produced in first 6 months of production
- Township - Location North and South of base line
- Range - Location East and West of meridian

In [5]:
# Create new wells data frame with only columns of interest and horizontal drill types.
wells1 = wells[['Well Name',
                'Well Number',
                'Operator Alias', 
                'Field',
                'Target Formation', 
                'Measured Depth (TD)',
                'True Vertical Depth', 
                'Drill Type',
                'Upper Perforation', 
                'Lower Perforation',
                'Gross Perforated Interval', 
                'Horizontal Length', 
                'Treatment Job Count (API10)',
                'Cum BOE', 
                'Cum Gas', 
                'Cum Oil', 
                'First 6 BOE', 
                'First Prod Date', 
                'Township', 
                'Range',
                     ]]
# Clean up column names to remove spaces
wells1 = wells1.rename(columns={'Well Name':'WellName',
                'Well Number':'WellNumber',
                'Operator Alias':'OperatorAlias', 
                'Field':'Field',
                'Target Formation':'TargetFormation', 
                'Measured Depth (TD)':'TotalDepth',
                'True Vertical Depth':'VerticalDepth', 
                'Drill Type':'DrillType',
                'Upper Perforation':'UpperPerforation', 
                'Lower Perforation':'LowerPerforation',
                'Gross Perforated Interval':'GrossPerforatedInterval', 
                'Horizontal Length':'HorizontalLength', 
                'Treatment Job Count (API10)':'TreatmentJobCount',
                'Cum BOE':'CumBOE', 
                'Cum Gas':'CumGas', 
                'Cum Oil':'CumOil', 
                'First 6 BOE':'First6BOE', 
                'First Prod Date':'FirstProdDate', 
                'Township':'Township', 
                'Range':'Range'})
hzwells = wells1[wells1['DrillType'] == 'H']
hzwells.head()

,WellName,WellNumber,OperatorAlias,Field,TargetFormation,TotalDepth,VerticalDepth,DrillType,UpperPerforation,LowerPerforation,GrossPerforatedInterval,HorizontalLength,TreatmentJobCount,CumBOE,CumGas,CumOil,First6BOE,FirstProdDate,Township,Range
API14,,,,,,,,,,,,,,,,,,,,
5123192590000,BRINGELSON,1-28X,PETERSON ENERGY OPERATING,NEW DAY,NIOBRARA,7630.0,5827.0,H,6185.0,7415.0,1230.0,NaN,1.0,474.0,0.0,474.0,448.0,1997-03-01,09N,58W
5005072210100,STATE MASSIVE,1H,CONOCO PHILLIPS,DJ HORIZONTAL NIOBRARA,NIOBRARA,17291.0,8008.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05S,65W
5005073270100,STATE CHALLENGER 5-65 2-3,3BYH,CONOCO PHILLIPS,WILDCAT,NIOBRARA,17847.0,7983.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05S,65W
5123343680100,BARRACUDA,29-11-7-60,VERDAD RESOURCES LLC,WILDCAT,NIOBRARA,10697.0,6351.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07N,60W
5123351000100,RAINDANCE FC,23-369HC,GREAT WESTERN OPERATING,WATTENBERG,CODELL,20186.0,6580.0,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06N,67W


Great - now only having the columns of interest, I can zoom in on the data that is relevant to this project to see what needs to be cleaned, what is missing, etc.

In [6]:
# Inspect columns
hzwells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15092 entries, 5123192590000 to 21636790000
Data columns (total 20 columns):
WellName                   15092 non-null object
WellNumber                 15092 non-null object
OperatorAlias              14312 non-null object
Field                      15092 non-null object
TargetFormation            15092 non-null object
TotalDepth                 14242 non-null float64
VerticalDepth              12006 non-null float64
DrillType                  15092 non-null object
UpperPerforation           4987 non-null float64
LowerPerforation           4986 non-null float64
GrossPerforatedInterval    4983 non-null float64
HorizontalLength           15068 non-null float64
TreatmentJobCount          6624 non-null float64
CumBOE                     6265 non-null float64
CumGas                     6233 non-null float64
CumOil                     6259 non-null float64
First6BOE                  5811 non-null float64
FirstProdDate              6328 non-n

Alright, it looks like there are about 6K entries that include the first 6 months of BOE, which will be my target output at least to start.  The other wells must be pre-production, didn't track this value, or did not produce.  Let's pull out these wells.

In [7]:
# Creating data frame with only producing wells
hzwells_pr = hzwells[hzwells['First6BOE'] >0]
print(hzwells_pr.shape)
hzwells_pr.head()

(5811, 20)


,WellName,WellNumber,OperatorAlias,Field,TargetFormation,TotalDepth,VerticalDepth,DrillType,UpperPerforation,LowerPerforation,GrossPerforatedInterval,HorizontalLength,TreatmentJobCount,CumBOE,CumGas,CumOil,First6BOE,FirstProdDate,Township,Range
API14,,,,,,,,,,,,,,,,,,,,
5123192590000,BRINGELSON,1-28X,PETERSON ENERGY OPERATING,NEW DAY,NIOBRARA,7630.0,5827.0,H,6185.0,7415.0,1230.0,NaN,1.0,474.0,0.0,474.0,448.0,1997-03-01,09N,58W
5001097530000,WEP,4-28-11-3-64,AXIS ENERGY,DJ HORIZONTAL NIOBRARA,NIOBRARA,11431.0,7551.0,H,7519.0,11431.0,3912.0,4376.37,2.0,28664.0,45552.0,21072.0,10662.0,2017-04-01,03S,64W
5001097540000,WEP,1-28-11-3-64,AXIS ENERGY,DJ HORIZONTAL NIOBRARA,NIOBRARA,11715.0,7594.0,H,8151.0,11715.0,3564.0,3575.27,2.0,33546.0,60174.0,23517.0,13185.0,2013-02-01,03S,64W
5001098010000,SHARP,24-3-11HC,WARD PETR,WATTENBERG,CODELL,10161.0,7774.0,H,NaN,NaN,NaN,2247.84,1.0,188242.0,273498.0,142659.0,46241.0,2015-01-01,01S,67W
5001098260000,ALICIA,12-15H-5N,WARD PETR,SPINDLE,NIOBRARA,13842.0,7821.0,H,NaN,NaN,NaN,5524.53,1.0,159344.0,348837.0,101205.0,53409.0,2015-09-01,01S,68W


#### Township and Range
Since these values need to go together for each individual well, I will first clean and then join them together.

In [8]:
hzwells_pr.Range.value_counts()

65W    1046
66W     865
67W     846
63W     631
64W     608
62W     502
68W     471
58W     387
59W     153
61W     147
57W      81
60W      72
52W       2
Name: Range, dtype: int64

These look clean, let's check out townships.

In [9]:
hzwells_pr.Township.value_counts()

06N    922
02N    900
05N    861
N      704
03N    661
04N    523
01N    464
09N    374
07N    275
08N     73
01S     24
04S     18
05S     10
03S      2
Name: Township, dtype: int64

Interesting, there are a significant number of unlabeled townships, with just the direction, north. After looking at all of the API numbers of wells with "N" townships, they all come from Weld County, where townships include 10N, 11N, and 12N. I will re-code these as 10+.

In [10]:
hzwells_pr.Township = hzwells_pr.Township.map(lambda x: '10+N' if x == 'N' else x)

Now that the townships and ranges are all cleaned up, they really go together to designate a single plot, so should not be individual variables.  I will concatenate them together. 

In [11]:
hzwells_pr['TR'] = hzwells_pr.Township + hzwells_pr.Range
hzwells_pr[['TR','Township','Range']].head()

,TR,Township,Range
API14,,,
5123192590000,09N58W,09N,58W
5001097530000,03S64W,03S,64W
5001097540000,03S64W,03S,64W
5001098010000,01S67W,01S,67W
5001098260000,01S68W,01S,68W


In [12]:
hzwells_pr.TR.nunique()

93

#### County
In addition to the 78 townships and ranges, I want to add in a county feature to identify the location of the wells.

In [13]:
county = []
for ind in hzwells_pr.index:
    inds = str(ind)
    cnty_no = int(inds[1:4])
    if cnty_no == 1:
        county.append('Adams')
    elif cnty_no == 5:
        county.append('Arapahoe')
    elif cnty_no == 69:
        county.append('Larimer')
    elif cnty_no == 123:
        county.append('Weld')
    else:
        county.append('Morgan')
                            
hzwells_pr['County'] = county
hzwells_pr.County.unique()

array(['Weld', 'Adams', 'Arapahoe', 'Larimer', 'Morgan'], dtype=object)

#### Target Formation
Next, let's take a look at the different target formations. This will tell us the type of rock that the well is targeting and should be important in the amount of production of the well.

In [14]:
hzwells_pr.TargetFormation.value_counts()

NIOBRARA                                        4406
CODELL                                           826
CODELL SAND                                      167
CODELL-FORT HAYS                                 123
CODELL-FORT HAYS-NIOBRARA                         92
CARLILE-CODELL-FORT HAYS                          62
CARLILE-CODELL-FORT HAYS-NIOBRARA                 53
CODELL-CARLILE                                    27
CODELL-NIOBRARA                                   13
FORT HAYS-NIOBRARA                                 9
FORT HAYS                                          6
GREENHORN                                          6
SUSSEX                                             5
NIOBRARA-SHARON SPRINGS                            4
NOT COMPLETED (UNKNOWN)                            2
NOT COMPLETED (NIOBRARA )                          2
O SAND                                             2
J SAND                                             2
COGCC FORMATION CODE: NBCDCL                  

The majority of wells targeted the Niobrara formation or Codell formation. According to industry professionals, the only two oil and gas producing formations are Niobrara and Codell, so the target formations listed with multiple different formations should really just be the deepest formation. The formations from top to bottom are: Niobrara, Fort Hays, Codell, and Carlisle. I will recode these as such.

In [15]:
hzwells_pr.TargetFormation = hzwells_pr.TargetFormation.map(lambda x: str(x))
hzwells_pr.TargetFormation = hzwells_pr.TargetFormation.map(lambda x: 'CODELL' if re.search('CODELL|CD', x) else x)
hzwells_pr.TargetFormation = hzwells_pr.TargetFormation.map(lambda x: 'NIOBRARA' if re.search('NIOBRARA', x) else x)
hzwells_pr.TargetFormation = hzwells_pr.TargetFormation.map(lambda x: x if re.search('CODELL|NIOBRARA', x) else 'OTHER')
hzwells_pr.TargetFormation.value_counts()

NIOBRARA    4421
CODELL      1365
OTHER         25
Name: TargetFormation, dtype: int64

Great! Now that I have all of that information captured, I will keep this Colorado data in a csv file in case I need it at a later time.

In [16]:
# Write DataFrame to csv
hzwells_pr.to_csv('HZwells.csv')

## FracFocus Data
The final set of data that I was provided was from a website called [FracFocus](http://fracfocusdata.org/), which contains data for each well and includes information about each individual ingredient used in the fracing process in wells across the United States.  I will compile these from the FracFocus folder from 14 different csv files into one data frame to then inspect and clean.

In [17]:
import glob
# Location of files
path = 'FracFocus'
allFiles = glob.glob(path + "/*.csv")
# Read together all files
fracfocus = pd.concat((pd.read_csv(f, low_memory=False) for f in allFiles))

Great, now let's see if this worked and check the shape of the file.

In [18]:
print(fracfocus.shape)
fracfocus.head()

(3271228, 39)


,UploadKey,JobStartDate,JobEndDate,APINumber,StateNumber,CountyNumber,OperatorName,WellName,Latitude,Longitude,...,IngredientKey,IngredientName,CASNumber,PercentHighAdditive,PercentHFJob,IngredientComment,IngredientMSDS,MassIngredient,ClaimantCompany,DisclosureKey
0,d2255659-ac30-4973-89ab-b6eb5c94f62d,10/9/2014 12:00:00 AM,10/10/2014 12:00:00 AM,42461396320000,42,461,"Parsley Energy Operations, LLC",Morgan 33-3,31.42074,-101.964586,...,e68b39b8-9c57-4cbe-a5ae-5422eabe1ee4,Diethanolamide,Proprietary,70.0,0.002441,None,True,340.559,NaN,d2255659-ac30-4973-89ab-b6eb5c94f62d
1,d2255659-ac30-4973-89ab-b6eb5c94f62d,10/9/2014 12:00:00 AM,10/10/2014 12:00:00 AM,42461396320000,42,461,"Parsley Energy Operations, LLC",Morgan 33-3,31.42074,-101.964586,...,89c33ca6-6637-4556-8a9f-8494c70abd6d,Potassium Chloride,7447-40-7,100.0,0.544779,None,True,76000.000,NaN,d2255659-ac30-4973-89ab-b6eb5c94f62d
2,d2255659-ac30-4973-89ab-b6eb5c94f62d,10/9/2014 12:00:00 AM,10/10/2014 12:00:00 AM,42461396320000,42,461,"Parsley Energy Operations, LLC",Morgan 33-3,31.42074,-101.964586,...,5aff3818-bd4f-421d-8863-008d136f6109,Polysaccharide Blend,Proprietary,60.0,0.004824,None,True,672.941,NaN,d2255659-ac30-4973-89ab-b6eb5c94f62d
3,d2255659-ac30-4973-89ab-b6eb5c94f62d,10/9/2014 12:00:00 AM,10/10/2014 12:00:00 AM,42461396320000,42,461,"Parsley Energy Operations, LLC",Morgan 33-3,31.42074,-101.964586,...,538c18dd-8d7f-4b59-ac8d-078dddda91d8,Petroleum Distillates,Proprietary,60.0,0.004824,None,True,672.941,NaN,d2255659-ac30-4973-89ab-b6eb5c94f62d
4,d2255659-ac30-4973-89ab-b6eb5c94f62d,10/9/2014 12:00:00 AM,10/10/2014 12:00:00 AM,42461396320000,42,461,"Parsley Energy Operations, LLC",Morgan 33-3,31.42074,-101.964586,...,8479c286-0273-4c0a-bf5c-189901bf0da8,Petroleum Distillates,Proprietary,25.0,0.020327,None,True,2835.798,NaN,d2255659-ac30-4973-89ab-b6eb5c94f62d


Awesome.  Now let's take a look at all of these columns.

In [19]:
fracfocus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3271228 entries, 0 to 249999
Data columns (total 39 columns):
UploadKey                  object
JobStartDate               object
JobEndDate                 object
APINumber                  int64
StateNumber                int64
CountyNumber               int64
OperatorName               object
WellName                   object
Latitude                   float64
Longitude                  float64
Projection                 object
TVD                        float64
TotalBaseWaterVolume       float64
TotalBaseNonWaterVolume    float64
StateName                  object
CountyName                 object
FFVersion                  int64
FederalWell                bool
IndianWell                 bool
Source                     float64
DTMOD                      float64
PurposeKey                 object
TradeName                  object
Supplier                   object
Purpose                    object
SystemApproach             float64
IsWa

Alright, there are a lot of extra columns that I won't need, but I need to extract the relevant information from each well from all of this information.  First, I will extract the different types of frac fluid that are being used in each well, along with potential ingredient purposes, as follows:
- Slickwater: friction, surfactant, mobility, and flow
- Gel: gel
- Crosslinked Gel: crosslinker, x-link, initiator, guar, catalyst

I will also identify the different ingredients indicative of the sand component, including proppant, sand, silicon, mesh, white, quartz, apatite, SiO2, aluminum oxide, titanium oxide, and iron oxide.

In [20]:
# Cleaning all purposes to be lowercase
fracfocus.Purpose = fracfocus.Purpose.map(lambda x: str(x).lower())
# Identifying all slickwater purpsoes
slick = 'friction|surfact|mobil|flow|emulsi'
fracfocus['slick'] = fracfocus.Purpose.map(lambda x: 1 if re.search(slick, x) else 0)
# Identifying all gel purposes
fracfocus['gel'] = fracfocus.Purpose.map(lambda x: 1 if re.search('gel', x) else 0)
# Identifying all crosslinked gel purposes
xlink = 'cross|x-link|initiator|guar|catalyst|viscos'
fracfocus['xlinkgel'] = fracfocus.Purpose.map(lambda x: 1 if re.search(xlink, x) else 0)

# Identifying all sand ingredients
fracfocus.IngredientName = fracfocus.IngredientName.map(lambda x: str(x).lower())
sand = 'proppa|propping|sand|sili|mesh|white|quartz|apatite|sio2|aluminum|titanium|iron'
fracfocus['sand'] = fracfocus.IngredientName.map(lambda x: 1 if re.search(sand, x) else 0)

In [21]:
# Inspecting the relevant columns
fracfocus[['slick','gel','xlinkgel','Purpose', 'IngredientName','sand']].head()

,slick,gel,xlinkgel,Purpose,IngredientName,sand
0,1,0,0,surfactant,diethanolamide,0
1,0,0,0,kcl potassium chloride,potassium chloride,0
2,0,1,0,liquid gellant,polysaccharide blend,0
3,0,1,0,liquid gellant,petroleum distillates,0
4,1,0,0,cationic friction reducer,petroleum distillates,0


Knowing that there are a lot of other ingredients in the frac fluid, this looks good.  Now let's use all of the ingredients that are sand to calculate the mass of sand used to frac the well. To calculate this, I will multiply the volume of water (given in gallons) by the density of water in pounds per gallon (8.34 pounds per gallon) to convert volume to pounds.  Then I will use that as the total mass of the fluid, and multiply by the percent by mass of each ingredient in decimal form to calulate the pounds of sand used for each sand ingredient.

$$ Mass_{ingredient(lbs)} =  Volume_{water(gal)} * density_{water (lbs/gal)} * \frac{Percent by mass_{ingredient}}{100%} $$

In [22]:
# Calculation of mass of sand used
fracfocus['sandmass'] = fracfocus.sand * fracfocus.TotalBaseWaterVolume * 8.34 * fracfocus.PercentHFJob / 100
# Checking mass calucations for sand ingredients
fracfocus[fracfocus.sand==1][['APINumber','IngredientName','TotalBaseWaterVolume','PercentHFJob', 'sandmass']].head()

,APINumber,IngredientName,TotalBaseWaterVolume,PercentHFJob,sandmass
15,42461396320000,silica,1480080.0,0.547811,6.762110e+04
21,42461396320000,silica,1480080.0,8.228315,1.015692e+06
26,42485455100000,silica,21428.0,11.674080,2.086269e+04
73,42495335600000,quartz,1075297.0,6.377023,5.718900e+05
76,43013526410000,crystalline silica (quartz),1198632.4,5.217009,5.215233e+05


Fantastic.  Now to understand each individual well, I need to combine all of the different ingredients by well to determine the different types of methods used per well.  Note that a well can have from one to three different types of fracing methods.  To do this, I will use the maximum value for each type of method and group by API number, so if a well contains any kind of slickwater ingredeients, for example, then the slick feature will have a 1, and if it doesn't, it will be a 0. 

In [23]:
# Creating separate data frame grouped by API number for frac method types
fracmethod = fracfocus.groupby('APINumber')[['APINumber','slick','gel','xlinkgel']].max()
# Setting index as API Number for joining purposes
fracmethod = fracmethod.reindex(fracmethod.APINumber).drop('APINumber', axis=1)

# Removing duplicate information between gel and crosslinked gel
for idx in fracmethod.index:
    if fracmethod.loc[idx, 'gel'] == 1 and fracmethod.loc[idx, 'xlinkgel'] == 1:
        fracmethod.loc[idx, 'gel'] = 0
    if fracmethod.loc[idx,'slick'] ==1 and (fracmethod.loc[idx, 'gel'] == 1 or fracmethod.loc[idx, 'xlinkgel'] == 1):
        fracmethod.loc[idx, 'hybrid'] = 1
    else:
        fracmethod.loc[idx, 'hybrid'] = 0
        
fracmethod.head()

,slick,gel,xlinkgel,hybrid
APINumber,,,,
4226932868,1,0,1,1.0
1007203590000,1,1,0,1.0
1007203600000,1,1,0,1.0
1007203610000,1,1,0,1.0
1007203620000,1,1,0,1.0


Super! That first API number looks like it is formatted incorrectly (without 14 digits), but knowing that this is not an API number for Colorado, I won't worry about it. For context, that first well was fraced with a crosslinked gel fluid, whereas the other four are only linear gel because there is an absence of crosslinker in the fluid. 

Now, let's do a similar extraction of information for the sand mass.  This time I will add together all of the sand components to get the total mass of sand by grouping by API number.

In [24]:
# Creating series with the total sand volume per APINumber
totsand = fracfocus.groupby('APINumber').sandmass.sum()
# Joining together sand mass and methods into one data frame for future states
fracmethod = fracmethod.join(totsand, how='left')

In [25]:
fracmethod.head()

,slick,gel,xlinkgel,hybrid,sandmass
APINumber,,,,,
4226932868,1,0,1,1.0,8552.338261
1007203590000,1,1,0,1.0,177493.467769
1007203600000,1,1,0,1.0,151188.848979
1007203610000,1,1,0,1.0,219070.601388
1007203620000,1,1,0,1.0,245962.571934


Let's briefly check out some of the wells that do not include sand volume.  If they are also missing the frac type I will have to drop them, as those are essential features to this data.

In [26]:
fracmethod[fracmethod.sandmass == 0][['slick','gel','xlinkgel','sandmass']].head()

,slick,gel,xlinkgel,sandmass
APINumber,,,,
1035202860000,1,1,0,0.0
1053207090000,1,0,1,0.0
1073217220000,0,0,0,0.0
1073217230000,0,0,0,0.0
1073217240000,0,0,0,0.0


In [27]:
# Number of non-zero sand volumes, which are likely to also have information about the frac method
fracmethod[fracmethod.sandmass != 0].shape

(76920, 5)

Alright, there are definitely wells for which I don't have information about the sand volume or type.  I will now make a final data frame that drops rows that are missing all four of those data points, imputes the average sand volume for that operator if just the sand volume is 0, and appends the data into the new, clean data frame.  Imputing the sand mass by operator is reasonable because most companies are consistent in their use of sand per well.

In [28]:
# Re-indexing data frame with API numbers
fracfocus_api = fracfocus.groupby('APINumber').first()

In [29]:
pos_sand = fracfocus[fracfocus.sandmass != 0]

avg_sand = pos_sand.groupby('OperatorName')['sandmass'].mean()

In [30]:
# Initializing new data frame
fracmethod_clean = pd.DataFrame()

for i in fracmethod.index:
    # Skipping rows with no sand or method data
    if fracmethod.sandmass[i] == 0 and fracmethod.slick[i] == 0 and fracmethod.gel[i] == 0 and fracmethod.xlinkgel[i] == 0:
        continue
    # Skipping rows with no method data
    elif fracmethod.slick[i] == 0 and fracmethod.gel[i] == 0 and fracmethod.xlinkgel[i] == 0:
        continue
    # Selecting rows that are missing sand mass
    elif fracmethod.sandmass[i] == 0:
        try:
            # Identifying the operator
            operator = fracfocus_api.OperatorName.loc[i]
            # Using the operator average as the mass of sand for that well
            fracmethod.sandmass.loc[i] = avg_sand[operator]
            # Appending row to new data frame
            fracmethod_clean = fracmethod_clean.append(fracmethod.loc[i])
        except:
            continue
    # Appending all wells with sand mass to new data frame
    else:
        fracmethod_clean = fracmethod_clean.append(fracmethod.loc[i])
fracmethod_clean.head()

,gel,hybrid,sandmass,slick,xlinkgel
4226932868,0.0,1.0,8552.338261,1.0,1.0
1007203590000,1.0,1.0,177493.467769,1.0,0.0
1007203600000,1.0,1.0,151188.848979,1.0,0.0
1007203610000,1.0,1.0,219070.601388,1.0,0.0
1007203620000,1.0,1.0,245962.571934,1.0,0.0


Great! Now let's check the shape of this new, clean data frame.

In [31]:
fracmethod_clean.shape

(89393, 5)

Good, I didn't lose that many rows of data and was able to impute the operator average to keep the information for the well. Finally, I will write this clean data frame to csv to perform further analyses.

In [32]:
# Writing to file for other states
#fracfocus.to_csv('fracfocus_clean.csv')
fracmethod.to_csv('fracmethod_clean.csv')

### Joining on Colorado Well Data
Finally, let's put this all together and join the horizontal well data with frac method and total sand volume on the API number indices and check the shape of the data frame.

In [33]:
# Joining the three items on the API number of the wells
hzfracwells = hzwells_pr.join(fracmethod, how='inner')
print(hzfracwells.shape)
hzfracwells.head()

(5220, 27)


,WellName,WellNumber,OperatorAlias,Field,TargetFormation,TotalDepth,VerticalDepth,DrillType,UpperPerforation,LowerPerforation,...,FirstProdDate,Township,Range,TR,County,slick,gel,xlinkgel,hybrid,sandmass
5001097530000,WEP,4-28-11-3-64,AXIS ENERGY,DJ HORIZONTAL NIOBRARA,NIOBRARA,11431.00,7551.00,H,7519.0,11431.0,...,2017-04-01,03S,64W,03S64W,Adams,0,0,0,0.0,0.000000e+00
5001097540000,WEP,1-28-11-3-64,AXIS ENERGY,DJ HORIZONTAL NIOBRARA,NIOBRARA,11715.00,7594.00,H,8151.0,11715.0,...,2013-02-01,03S,64W,03S64W,Adams,0,0,0,0.0,0.000000e+00
5001098010000,SHARP,24-3-11HC,WARD PETR,WATTENBERG,CODELL,10161.00,7774.00,H,NaN,NaN,...,2015-01-01,01S,67W,01S67W,Adams,1,0,1,1.0,2.533881e+06
5001097850000,STATE OF CO,1S-66-36-1609CH,EXTRACTION O&G,THIRD CREEK,NIOBRARA,12177.31,7576.73,H,NaN,NaN,...,2014-06-01,01S,66W,01S66W,Adams,1,0,1,1.0,3.274332e+06
5001097830000,STATE OF CO,1S-66-36-0108BH,EXTRACTION O&G,THIRD CREEK,NIOBRARA,12164.98,7511.68,H,NaN,NaN,...,2014-06-01,01S,66W,01S66W,Adams,1,0,1,1.0,3.045143e+06


Great! It looks like we have about 5K Colorado wells with their relevant information, including the type of the well and the volume of sand used.  Although a couple look like they're missing this data, which, upon further inspection is likely from the extraction of information for the frac focus registry where some of the information was not consistently formatted to be brought into the registry in the first place. 

Let's take a look at the columns.

In [34]:
hzfracwells.describe()

,TotalDepth,VerticalDepth,UpperPerforation,LowerPerforation,GrossPerforatedInterval,HorizontalLength,TreatmentJobCount,CumBOE,CumGas,CumOil,First6BOE,slick,gel,xlinkgel,hybrid,sandmass
count,5212.000000,5210.000000,4319.000000,4319.000000,4319.000000,5218.000000,5218.000000,5220.000000,5.220000e+03,5220.000000,5220.000000,5220.000000,5220.000000,5220.000000,5220.000000,5.218000e+03
mean,12878.990846,6857.418305,7386.293355,12740.042602,5353.749248,6011.867813,1.311614,155626.998276,4.177970e+05,85994.121073,56920.921839,0.864943,0.024330,0.608429,0.631609,3.787006e+06
std,2262.604223,554.396400,634.410077,2109.403861,2034.062398,2231.362238,0.518272,84886.020930,3.149566e+05,44536.048822,29003.069779,0.341818,0.154085,0.488148,0.482414,4.267373e+06
min,2839.000000,713.000000,248.000000,6870.000000,49.000000,29.590000,1.000000,402.000000,6.350000e+02,296.000000,137.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,11310.000000,6543.500000,7095.500000,11255.000000,3962.000000,4483.465000,1.000000,95439.500000,1.799692e+05,55849.500000,37518.750000,1.000000,0.000000,0.000000,0.000000,1.826881e+06
50%,12052.000000,6992.000000,7501.000000,11982.000000,4370.000000,4933.265000,1.000000,140872.500000,3.284300e+05,77235.500000,52869.500000,1.000000,0.000000,1.000000,1.000000,3.163582e+06
75%,14206.000000,7235.327500,7748.000000,13944.000000,6560.000000,7377.195000,2.000000,197461.750000,5.858402e+05,107268.500000,70720.250000,1.000000,0.000000,1.000000,1.000000,4.548139e+06
max,22086.000000,9793.000000,16864.000000,21986.000000,15982.000000,15695.510000,5.000000,677112.000000,2.716674e+06,356272.000000,236728.000000,1.000000,1.000000,1.000000,1.000000,1.681255e+08


Again, we're missing some information about perforations, and it looks like we could be missing about 25% of the sand volume data.  I will check that out after I know that perforations are significant to my analysis.  I will keep them for now.

Finally, I will write this clean data frame to csv to perform further analyses.

In [35]:
# Write DataFrame to csv
hzfracwells.to_csv('fracwells_co.csv', index_label='APINumber')